<a href="https://colab.research.google.com/github/tluxxx/vbt_basics/blob/main/vbt_test_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction into vectorbt - part 2
* valid for latest free version (version: 0.28.1)
* portfolio with one ticker

Content:
* Trade prices (at entry and exit)
    * default trade prices
    * next bars's Open as trade price
* Impact of parameters:
    * `size_type`, `size` and `min_size`
    * `fees` and `slippage`
* StopLoss and TakeProfit



In [1]:
!pip install vectorbt ta python-dotenv --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.7/420.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00


In [2]:
import vectorbt as vbt
import pandas as pd
import numpy as np
from numba import njit

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import yfinance as yf
import ta

## 0. helpers functions

In [4]:
import sys
import importlib
import os
import helper

from google.colab import drive
from dotenv import load_dotenv

# connnecting Google-Drive, loading .env-file and getting path_data
from pathlib import Path
drive.mount('/content/drive')
load_dotenv('/content/drive/My Drive/.env')
path_data = os.getenv('PATH_VBT')               # replace it with your path
sys.path.append(path_data)

# importing supporting functions (from helper-module in path_data)
for mod in ['timelines', 'entries_exits', 'plotting_support']:
    importlib.reload(importlib.import_module(f'helper.{mod}'))

from helper.timelines import get_timeline_basic, get_timeline_extended
from helper.entries_exits import lense_at_entry_exit, classify_exit_types
from helper.plotting_support import _is_boolean_series, plot_multi_subplot_trading_data

Mounted at /content/drive


## 1. Data Download and Signal Generation
We apply the same methodology used in Part 1:
* single ticker portfolio
* Download price data from **yfinance**
* Use an **SMA**-crossing, LongOnly strategy

* Additionally, we extract **all** OHLC price-components into separate `Pandas.Series` for later use, rather than relying only on close prices only, as in Part 1.

In [5]:
# download data
ticker = 'AAPL'
start_date = '2022-01-01'
end_date = '2025-10-11'
prices = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)

# preparing for ta & vbt-use (remove MultiLevel-column-index, transform to several pd.Series)
close = prices.Close[ticker]
open = prices.Open[ticker]
high = prices.High[ticker]
low = prices.Low[ticker]

# entry and exit signals via ta,
fast_window = 10
slow_window = 60

fast_sma = ta.trend.sma_indicator(close, window=fast_window)
slow_sma = ta.trend.sma_indicator(close, window=slow_window)

# entry and exit signals by simple SMA-crossing
entries = ((fast_sma > slow_sma) & (fast_sma.shift(1) < slow_sma.shift(1))).rename(ticker)
exits = ((fast_sma < slow_sma) & (fast_sma.shift(1) > slow_sma.shift(1))).rename(ticker)


[*********************100%***********************]  1 of 1 completed


## 2. Default trade-prices for entries and exits and default order size
We start with the baseline-portfolio, already introduced in part 1.

In [6]:
# baseline portfolio
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    exits,                  # series (boolean) of exit-signals
    init_cash=10_000,       # initial cash
    freq='D',               # all parameters on daily base
)

Next topic is a tabulation of:
* executed trades and their key parameters
* generation of a timeline (=dataframe with all portfolio parameters of interest)
* inspecting the timeline around entries and exits.

The first topic can be covered using the built-in `pf.trades.records_readable`-method.

For the second topic, we generated the function `get_timeline_basic(....)` stored in our helper module.

The timeline-dataframe will contain:
* dates (index of the data-frame)
* OHLC-price data
* (raw) entry and exit-signals (True if signal is triggered)
* effective entries and exits, generated from the table of trades, true for effective signals (= signals that are executed, i.e. results in an entry/exit of a position)

Note: Why do we need effective entries and exits? Because not all (raw) entries and exits signals will be executed. Examples are:
* If we already Long and miss the required cash, an entry signal is ignored.
* If we are already Flat an exit-signal will be ignored.




We can add now to this timeline-dataframe other parameters, as needed for our analyses.

In our case we will add the values of the fast-SMA/slow-SMA and the value of e cash for each timestep.

We generate now a dataframe for all trades with key data and the timeline-dataframe as follows:


In [7]:
# key trade data
trades = pf.trades.records_readable

# timeline for standard + specific trading parameter
timeline = get_timeline_basic(prices, entries, exits, trades)   # standards (price, entry, exits)
timeline['sma_fast'] = fast_sma                                 # Adding fast-SMA & slow-SMA
timeline['sma_slow'] = slow_sma
timeline['cash'] = pf.cash()                                    # adding cash

The timeline can now be inspected by slicing the DataFrame for relevant periods using .loc or .iloc.

As an alternative, I created a helper function `lense_at_entry_exit(...)` that displays a “lens” view of the timeline around a selected entry and exit date—showing only the window from n bars before to n bars after each event.




Now we tabulate the trade-table and the lense around the first entry/exit. We can easily observe:
* Entry:  
    * an entry signal is triggered at bar [i]
    * close [i] is used as entry-price
    * the position size [i+1] equals cash [i-1] / close [i], i.e. all available cash is used to execute the order.
* Exit:
    * an exit signal is triggered ast bar [j]
    * close [j] is used as exit-price
    * the position is fully closed, cash [j+1] is increased by the earnings of the exit-order.    

By default, vbt uses the close price of a bar, where an effective signal is triggered, for executing the trade. Since the signal itself is generated at the close, this is a somewhat theoretical scenario.  

In [8]:
# trades and key trade data
n = 3               # number of trades to be displayed
print(f'xxx  first {n} trades with key trade data')
display(trades.head(n))
print("\n")

# timeline around entry and exit of the first trade
entry_date = '2022-07-19'
exit_date = '2022-09-16'
n_r = 1             # number of rows before/after entry/exit
lense_at_entry_exit(timeline, entry_date, exit_date, n_r)

xxx  first 3 trades with key trade data


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,67.429314,2022-07-19,148.303452,0.0,2022-09-16,148.214432,0.0,-6.002542,-0.000600,Long,Closed,0
1,1,AAPL,67.152777,2022-11-23,148.824783,0.0,2022-12-07,138.845367,0.0,-670.145488,-0.067055,Long,Closed,1
2,2,AAPL,66.185399,2023-01-30,140.874756,0.0,2023-08-11,175.894928,0.0,2317.824076,0.248591,Long,Closed,2




### Timeline around Entry Date: 2022-07-19 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits,sma_fast,sma_slow,cash
Date,,,,,,,,,,,
2022-07-18,148.048109,148.863289,144.080247,144.443649,False,False,False,False,143.373117,143.633373,10000.0
2022-07-19,145.278452,148.529340,144.286494,148.303452,True,False,True,False,144.300256,143.384960,0.0
2022-07-20,148.421311,150.974887,147.684704,150.307022,False,False,False,False,145.294182,143.245616,0.0




### Timeline around Exit Date: 2022-09-16 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits,sma_fast,sma_slow,cash
Date,,,,,,,,,,,
2022-09-15,152.099266,152.679546,148.883211,149.856873,False,False,False,False,153.529294,153.373494,0.000000
2022-09-16,148.716030,148.853720,145.922860,148.214432,False,True,False,True,152.815268,153.628186,9993.997458
2022-09-19,146.847341,152.010750,146.640814,151.932068,False,False,False,False,152.684462,153.897040,9993.997458


## 3. Using next bar's  Open for trading
Let's discuss a more realistic approach. The signal is generated at the close of a certain bar. It is more realistic to assume, that the trading is executed at the open of the next bar. This mimics the behaviour of a part-time stock trader, using daily-charts, who does his analyses after markets-closure and places market orders for the next day. In a fairly liquid market, the order will than be filled next day around open.

This (more realistic) behaviour can be modelled in vbt by setting the parameter `price` to the series 'open' (but shifting the open by one bar back).

This portfolio instance is generated as follows:

In [9]:
# baseline portfolio
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    exits,                  # series (boolean) of exit-signals
    price=open.shift(-1),   # price to consider for trading (here: execution at next bar's open)
    init_cash=10_000,       # inital cash
    freq='D',               # all parameters on daily base
)

We can clearly observe, that the positions are opened and closed at next bars open.

In [10]:
# key trade data
trades = pf.trades.records_readable

# timeline for standard + specific trading parameter
timeline = get_timeline_basic(prices, entries, exits, trades)    # standards (price, entry, exits)
timeline['sma_fast'] = fast_sma                                 # adding fast-SMA & slow-SMA
timeline['sma_slow'] = slow_sma
timeline['cash'] = pf.cash()                                    # adding cash

# display of trades and key trade data
n = 3               # number of trades to be displayed
print(f'xxx  first {n} trades with key trade data')
display(trades.head(n))
print("\n")

# timeline around entry and exit of the first trade
entry_date = '2022-07-19'
exit_date = '2022-09-16'
n_r = 1             # number of rows before/aftr entry/exit
lense_at_entry_exit(timeline, entry_date, exit_date, n_r)


xxx  first 3 trades with key trade data


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,67.375769,2022-07-19,148.421311,0.0,2022-09-16,146.847341,0.0,-106.047410,-0.010605,Long,Closed,0
1,1,AAPL,67.717723,2022-11-23,146.105808,0.0,2022-12-07,140.244239,0.0,-396.932068,-0.040119,Long,Closed,1
2,2,AAPL,67.556391,2023-01-30,140.579158,0.0,2023-08-11,176.073011,0.0,2397.836629,0.252483,Long,Closed,2




### Timeline around Entry Date: 2022-07-19 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits,sma_fast,sma_slow,cash
Date,,,,,,,,,,,
2022-07-18,148.048109,148.863289,144.080247,144.443649,False,False,False,False,143.373117,143.633373,10000.0
2022-07-19,145.278452,148.529340,144.286494,148.303452,True,False,True,False,144.300256,143.384960,0.0
2022-07-20,148.421311,150.974887,147.684704,150.307022,False,False,False,False,145.294182,143.245616,0.0




### Timeline around Exit Date: 2022-09-16 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits,sma_fast,sma_slow,cash
Date,,,,,,,,,,,
2022-09-15,152.099266,152.679546,148.883211,149.856873,False,False,False,False,153.529294,153.373494,0.00000
2022-09-16,148.716030,148.853720,145.922860,148.214432,False,True,False,True,152.815268,153.628186,9893.95259
2022-09-19,146.847341,152.010750,146.640814,151.932068,False,False,False,False,152.684462,153.897040,9893.95259


Note: We have demonstrated how to generate timeline DataFrames containing OHLC data, entry/exit signals, effective entries/exits, and how to extend the timeline with additional parameters of interest (e.g., indicators).

To keep the discussion concise, the upcoming sections will not include screenshots of trades or timelines. Instead, we will focus on compact descriptions of the portfolio’s behavior. You are encouraged to inspect the code and print-outs on GitHub for verification.

## 4. Impact of Fees and Slippage.
Lets start wtiht Fees.

**Fee**: We can include fixed fees (fee per trade) and/or variable fee (in % of the position). These parameters reflects the costs, associated with our trading.

In [11]:
# portfolio with fees (but no slippage)
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    exits,                  # series (boolean) of exit-signals
    price=open.shift(-1),   # price to consider for trading (ensures execution at next open)
    fees=0.0025,            # 0.25% per order
    fixed_fees=4.5,         # 4.5 EUR/order
    slippage=0,             # no slippage
    init_cash=10_000,       # inital cash
    freq='D',               # all parameters on daily base
)

We can now compare the trade table with the one, generated without fees and observe:

* The entry and exit prices still match the relevant open prices in the timeline (since `price` = open.shift(-1)).

* The PnL values differ: fees are deducted from the profit/loss.

Note: As in real life, fees are applied per order—separately for entry and exit.

In [12]:
# trades and key data
trades = pf.trades.records_readable

print(f'xxx  first {n} trades with key trade data')
display(trades.head(n))
print("\n")

xxx  first 3 trades with key trade data


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,67.177506,2022-07-19,148.421311,29.426434,2022-09-16,146.847341,29.162096,-164.323880,-0.016481,Long,Closed,0
1,1,AAPL,67.120258,2022-11-23,146.105808,29.016649,2022-12-07,140.244239,28.033074,-450.479708,-0.045936,Long,Closed,1
2,2,AAPL,66.562521,2023-01-30,140.579158,27.893258,2023-08-11,176.073011,33.799659,2300.867447,0.245890,Long,Closed,2


Slippage

In a realistic simulation, we also need to account for slippage, expressed as a percentage of the entry/exit price. Slippage reflects the fact that trades are rarely executed exactly at the intended price, often due to limited liquidity.

To model this, a “penalty” is applied to the trade price:
* Long entry: price is increased by 1 + slippage
* Long exit: price is reduced by 1 - slippage

For highly liquid stocks, a slippage of 0.05% (0.0005) is usually adequate.

In [13]:
# Portfolio with slippage, no fees
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    exits,                  # series (boolean) of exit-signals
    price=open.shift(-1),   # price to consider for trading (ensures execution at next open)
    fees=0,                 # no fee
    fixed_fees=0,           # no fee
    slippage=0.01,          # 1% (just to make the impact clear)
    init_cash=10_000,       # inital cash
    freq='D',               # all parameters on daily base
)


We can clearly observe the impact of slippage by comparing with the baseline portfolio:
* Entry prices are higher
* Exit prices are lower

As a result, the PnL is reduced, compared to the baseline.

In [14]:
# trades and key data
trades = pf.trades.records_readable

print(f'xxx  first {n} trades with key trade data')
display(trades.head(n))
print("\n")

xxx  first 3 trades with key trade data


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,66.708682,2022-07-19,149.905524,0.0,2022-09-16,145.378868,0.0,-301.967264,-0.030197,Long,Closed,0
1,1,AAPL,65.719582,2022-11-23,147.566866,0.0,2022-12-07,138.841797,0.0,-573.407883,-0.059126,Long,Closed,1
2,2,AAPL,64.264733,2023-01-30,141.984949,0.0,2023-08-11,174.312281,0.0,2077.507337,0.227681,Long,Closed,2


## 5. The role of size, size_type and `min_size`
By default, `size_type` is set to 'Percent' and `size` is set to 1.0. Therefore for a effective entry signal always 100% of the **available cash**  is used, a effective exit-signal closes the position completely. See also the section above with details of our baseline-portfolio.

Below we will generate the portfolio instance and we will record the portfolio-value, the portfolio-cash and the number of assets in our portfolio.

In [15]:
# standard settings (size_type ='Percent' and size=1.0)
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    exits,                  # series (boolean) of exit-signals
    size_type='Percent',    # order volume defined by percent of available cash
    size=1.0,               # 100% to be used
    init_cash=10_000,       # initial cash
    freq='D',               # all parameters on daily base
)

# recording results
value_100_pct = pf.value()
cash_100_pct = pf.cash()
assets_100_pct = pf.assets()

The parameter `size` controls the percentage of cash used, if we enter into a position.

If we set `size` to 0.3 this means:
* at an effective entry signal, 30% of the available cash is used for entry-order.
* at an effective exit signal the position is completely closed

For this portfolio instance we will record the value, the cash and the number of assets after generation.

In [16]:
# modified settings (size_type ='Percent' and size=0.3)
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    exits,                  # series (boolean) of exit-signals
    size_type='Percent',    # order volume defined by percent of available cash
    size=0.3,               # 30% to be used
    init_cash=10_000,       # initial cash
    freq='D',               # all parameters on daily base
)

# recording results
value_30_pct = pf.value()
cash_30_pct = pf.cash()
assets_30_pct = pf.assets()


If we plot the portfolio value, cash, and number of assets for the two portfolios, the differences become immediately clear:

In [17]:
# Create subplots for Cash, Assets, and Portfolio Value
fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.1,
                    subplot_titles=("Cash Evolution", "Assets Evolution", "Portfolio Value Evolution"))

# Add traces
fig.add_trace(go.Scatter(x=cash_100_pct.index, y=cash_100_pct, mode='lines', name='Cash (size = 1.0)', legendgroup='cash'),
              row=1, col=1)
fig.add_trace(go.Scatter(x=cash_30_pct.index, y=cash_30_pct, mode='lines', name='Cash (size = 0.3)', legendgroup='cash'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=assets_100_pct.index, y=assets_100_pct, mode='lines', name='Assets (size = 1.0)', legendgroup='assets', showlegend=True),
              row=2, col=1)
fig.add_trace(go.Scatter(x=assets_30_pct.index, y=assets_30_pct, mode='lines', name='Assets (size=0.3)', legendgroup='assets', showlegend=True),
              row=2, col=1)

fig.add_trace(go.Scatter(x=value_100_pct.index, y=value_100_pct, mode='lines', name='Value (size = 1.0)', legendgroup='value', showlegend=True),
              row=3, col=1)
fig.add_trace(go.Scatter(x=value_30_pct.index, y=value_30_pct, mode='lines', name='Value (size = 0.3)', legendgroup='value', showlegend=True),
              row=3, col=1)

# finetuning
fig.update_layout(height=600, width=1200,
                  showlegend=True, hovermode='x unified',
                  title_text="Comparison of Portfolio Evolution for different settings of `size`"
)

fig.update_yaxes(title_text="Cash", row=1, col=1)
fig.update_yaxes(title_text="Assets", row=2, col=1)
fig.update_yaxes(title_text="Value", row=3, col=1)

fig.show()

Other options for size_type are:

* `size_type` = 'Amount' – the size parameter defines the number of assets (shares) to be traded
* `size_type` = 'Value' – the size parameter defines the volume (=total entry value) of the trade

We choose `size_type` = 'Value'. We remember, that size in the trade-table denotes the number of assets. Therefore we expect as size the volume of the entry order (=parameter `size` in the `.from_signals()` class method) / Avg Entry Price. Thats exactly what you see.

In [18]:
# modified settings (size_type ='Value' and size=1_000)
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    exits,                  # series (boolean) of exit-signals
    size_type='Value',      # order volume defined by value
    size=1_000,             # value per (entry) order
    init_cash=10_000,       # initial cash
    freq='D',               # all parameters on daily base
)

In [19]:
# trades and key data
trades = pf.trades.records_readable

# trades and key trade data
n = 3               # number of trades to be displayed
print(f'xxx  first {n} trades with key trade data')
display(trades.head(n))
print("\n")

xxx  first 3 trades with key trade data


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,6.742931,2022-07-19,148.303452,0.0,2022-09-16,148.214432,0.0,-0.600254,-0.000600,Long,Closed,0
1,1,AAPL,6.719311,2022-11-23,148.824783,0.0,2022-12-07,138.845367,0.0,-67.054799,-0.067055,Long,Closed,1
2,2,AAPL,7.098504,2023-01-30,140.874756,0.0,2023-08-11,175.894928,0.0,248.590827,0.248591,Long,Closed,2


Now we set `size_type` to 'Amount' and select a number (e.g. `size`=5.0). As a result, each opening of a position is done with a number of assets = `size`.

In [20]:
# modified settings (size_type ='Amount' and size=5.0)
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    exits,                  # series (boolean) of exit-signals
    size_type='Amount',     # order volume defined by nb of assets
    size=5.0,               # number of assets traded at entry
    init_cash=10_000,       # initial cash (EUR, USD, ...)
    freq='D',               # all parameters on daily base
)

In [21]:
# trades and key data
trades = pf.trades.records_readable

n = 3               # number of trades to be displayed
print(f'xxx  first {n} trades with key trade data')
display(trades.head(n))
print("\n")

xxx  first 3 trades with key trade data


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,5.0,2022-07-19,148.303452,0.0,2022-09-16,148.214432,0.0,-0.445099,-0.000600,Long,Closed,0
1,1,AAPL,5.0,2022-11-23,148.824783,0.0,2022-12-07,138.845367,0.0,-49.897079,-0.067055,Long,Closed,1
2,2,AAPL,5.0,2023-01-30,140.874756,0.0,2023-08-11,175.894928,0.0,175.100861,0.248591,Long,Closed,2


Now lets explore about the roles of the parameters `min_size` (and also `max_size`).  As a reference we will generate a portfolio-instance using `size_type`='Percent' and `size`=0.3.

In [22]:
# reference case : size_type ='Percent' and size=0.3
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    exits,                  # series (boolean) of exit-signals
    size_type='Percent',    # order volume defined by percent of available cash
    size=0.3,               # 30% to be used
    init_cash=10_000,       # initial cash
    freq='D',               # all parameters on daily base
)

In [23]:
# listing all trdes in the reference portfolio
trades = pf.trades.records_readable
trades

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,20.228794,2022-07-19,148.303452,0.0,2022-09-16,148.214432,0.0,-1.800763,-0.000600,Long,Closed,0
1,1,AAPL,20.154303,2022-11-23,148.824783,0.0,2022-12-07,138.845367,0.0,-201.128171,-0.067055,Long,Closed,1
2,2,AAPL,20.863364,2023-01-30,140.874756,0.0,2023-08-11,175.894928,0.0,730.638601,0.248591,Long,Closed,2
3,3,AAPL,17.103754,2023-11-10,184.656128,0.0,2024-01-12,184.180618,0.0,-8.133000,-0.002575,Long,Closed,3
4,4,AAPL,16.321739,2024-01-23,193.353973,0.0,2024-02-06,187.528992,0.0,-95.073828,-0.030126,Long,Closed,4
5,5,AAPL,17.253269,2024-05-08,181.261353,0.0,2024-09-16,215.109390,0.0,583.989296,0.186736,Long,Closed,5
6,6,AAPL,14.606738,2024-09-24,226.097549,0.0,2024-11-11,223.220444,0.0,-42.025129,-0.012725,Long,Closed,6
7,7,AAPL,14.431527,2024-11-20,227.968948,0.0,2025-01-17,228.944534,0.0,14.079195,0.004279,Long,Closed,7
8,8,AAPL,13.376885,2025-02-24,246.257919,0.0,2025-03-07,238.255295,0.0,-107.050188,-0.032497,Long,Closed,8
9,9,AAPL,15.387482,2025-07-02,211.993668,0.0,2025-10-10,245.032471,0.0,508.383997,0.155848,Long,Open,9


The portfolio with `min_size`and `max_size` is generated as follows:

In [24]:
# modified settings (size_type ='Value' and size=3_000, min_size and max_size)
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    exits,                  # series (boolean) of exit-signals
    size_type='Percent',    # order volume defined by percent of available cash
    size=0.3,               # 30% to be used
    min_size = 16.0,        # minimum 16.0 assets to be traded
    max_size= 20.0,         # maximum 20.0 assets to be traded
    init_cash=10_000,       # initial cash (EUR, USD, ...)
    freq='D',               # all parameters on daily base
)



For `size_type` = 'Percent', 'Amount' or 'Value', every order is executed as long as cash is available for at least `min_size` **number(!!)** of assets.

Note: `min_size` is **always** expressed as **number of assets**, regardless, what `size_type`is selected! By inspecting the trade-table, you easily observe that some orders are not executed anymore.  By default `min_size`is set to 1e-8.

By setting `max_size` to a value, you can limit the number of assets in an order. Therefore all orders are cut-off at maximum `max_size` of assets.

In [ ]:
# trades and key data
trades = pf.trades.records_readable
trades

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,20.000000,2022-07-19,148.303467,0.0,2022-09-16,148.214432,0.0,-1.780701,-0.000600,Long,Closed,0
1,1,AAPL,20.000000,2022-11-23,148.824783,0.0,2022-12-07,138.845352,0.0,-199.588623,-0.067055,Long,Closed,1
2,2,AAPL,20.000000,2023-01-30,140.874741,0.0,2023-08-11,175.894913,0.0,700.403442,0.248591,Long,Closed,2
3,3,AAPL,17.057166,2023-11-10,184.656128,0.0,2024-01-12,184.180634,0.0,-8.110587,-0.002575,Long,Closed,3
4,4,AAPL,16.277282,2024-01-23,193.353973,0.0,2024-02-06,187.528976,0.0,-94.815115,-0.030126,Long,Closed,4
5,5,AAPL,17.206274,2024-05-08,181.261353,0.0,2024-09-16,215.109390,0.0,582.398617,0.186736,Long,Closed,5


## 6. TakeProfitStops


First, we will study TakeProfitStops in isolation (by excluding system based exits).

We can control the TakeProfitStop-Limit by setting:
* `tp_stop`= value

Value defines the Percentage of trade-profit of a position (value=0.15 = 15% profit-limit). In our example, close price is used as price-base.  

In [25]:
# generate a portfolio instance only with take-profit stops
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    # exits,                # no exits from the system
    tp_stop=0.15,           # take profit at 15%
    init_cash=10_000,       # initial cash
    freq='D',               # all parameters on daily base
)

When we inspect the timelines around the entries and exits of a trade, we observe:
* For Entries:
    * entry signal, triggered at bar[i]
    * entry date = date of bar[i]
    * entry price is close[i] (this bar's close)
    * under the hood: take-profit-limit is calulated (1 + `tp_stop`) * close [i] and ìs fixed for the trade
* For Exits:
    * if close [j] exceeds the take-profit-limit for the trade, the take profit exit is triggered at bar[j]
    * exit_date is date of bar[j]
    * exit price = close[j]

Note: The close [j] might be higher than the take-profit-limit, therefore we will not exaclty meet the `tp_stop`-value.

In [26]:
# trades and key data
trades = pf.trades.records_readable

# generating timeline for standard + specific trading parameter
timeline = get_timeline_basic(prices, entries, exits, trades)    # standards (price, entry, exits)

# display of trades and key trade data
n = 3               # number of trades to be displayed
print(f'xxx  first {n} trades with key trade data')
display(trades.head(n))
print("\n")

# timeline around entry and exit of the first trade
entry_date = '2022-07-19'
exit_date = '2022-08-17'
n_r = 1             # number of rows before/after entry/exit
lense_at_entry_exit(timeline, entry_date, exit_date, n_r)

xxx  first 3 trades with key trade data


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,67.429314,2022-07-19,148.303452,0.0,2022-08-17,171.671066,0.0,1575.662232,0.157566,Long,Closed,0
1,1,AAPL,77.780474,2022-11-23,148.824783,0.0,2023-05-05,171.251480,0.0,1744.359112,0.150692,Long,Closed,1
2,2,AAPL,72.134196,2023-11-10,184.656128,0.0,2024-06-13,212.794724,0.0,2029.754972,0.152384,Long,Closed,2




### Timeline around Entry Date: 2022-07-19 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits
Date,,,,,,,,
2022-07-18,148.048109,148.863289,144.080247,144.443649,False,False,False,False
2022-07-19,145.278452,148.529340,144.286494,148.303452,True,False,True,False
2022-07-20,148.421311,150.974887,147.684704,150.307022,False,False,False,False




### Timeline around Exit Date: 2022-08-17 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits
Date,,,,,,,,
2022-08-16,169.930240,170.844909,168.828718,170.176117,False,False,False,False
2022-08-17,169.920426,173.244668,169.723728,171.671066,False,False,False,True
2022-08-18,170.884252,172.015279,170.264638,171.277649,False,False,False,False


Let’s now use the next bar’s open price for trading and examine the behavior of the TakeProfit stop under this execution model.

In [27]:
# generate a portfolio instance only with take-profit stops
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    # exits,                # no exits from the system
    price=open.shift(-1),   # price to consider for trading (ensures execution at next open)
    tp_stop=0.15,           # take profit at 15%
    init_cash=10_000,       # initial cash
    freq='D',               # all parameters on daily base
)

When we inspect the timelines around entries and exits of a trade, we observe the following:

Entries

* Entry signal is triggered at bar [i]
* Entry date = date of bar [i]
* Entry price = open [i+1] (next bar’s open)
* Under the hood: the take-profit limit is calculated as (1 + tp_stop) * open [i+1] and is fixed for the trade

Exits
* If close [j] exceeds the take-profit limit, the take-profit exit is triggered at bar [j]
* Exit date = date of bar [j]
* Exit price = close [j]

Note: Even though we set price = open.shift(-1), stops are executed at the close of the bar, not at the next bar’s open.
Also, close [j] may exceed the take-profit limit, so the exact tp_stop value may not be precisely met.

In [28]:
# trades and key data
trades = pf.trades.records_readable

# generating timeline for standard + specific trading parameter
timeline=get_timeline_basic(prices, entries, exits, trades)    # standards (price, entry, exits)

# display trades and key trade data
n = 3               # number of trades to be displayed
print(f'xxx  first {n} trades with key trade data')
display(trades.head(n))
print("\n")

# display timeline around entry and exit of the first trade
entry_date = '2022-07-19'
exit_date = '2022-08-17'
n_r = 1             # number of rows before/aftr entry/exit
lense_at_entry_exit(timeline, entry_date, exit_date, n_r)

xxx  first 3 trades with key trade data


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,67.375769,2022-07-19,148.421311,0.0,2022-08-17,171.671066,0.0,1566.470144,0.156647,Long,Closed,0
1,1,AAPL,79.165027,2022-11-23,146.105808,0.0,2023-05-05,171.251480,0.0,1990.657838,0.172106,Long,Closed,1
2,2,AAPL,73.647401,2023-11-10,184.081552,0.0,2024-06-13,212.794724,0.0,2114.650453,0.155981,Long,Closed,2




### Timeline around Entry Date: 2022-07-19 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits
Date,,,,,,,,
2022-07-18,148.048109,148.863289,144.080247,144.443649,False,False,False,False
2022-07-19,145.278452,148.529340,144.286494,148.303452,True,False,True,False
2022-07-20,148.421311,150.974887,147.684704,150.307022,False,False,False,False




### Timeline around Exit Date: 2022-08-17 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits
Date,,,,,,,,
2022-08-16,169.930240,170.844909,168.828718,170.176117,False,False,False,False
2022-08-17,169.920426,173.244668,169.723728,171.671066,False,False,False,True
2022-08-18,170.884252,172.015279,170.264638,171.277649,False,False,False,False


## 7. Advanded TakeProfitStops

It looks somehow artificial, to wait for executing take profit stops to the end of the bar and to execute it at the close.

In real life you know the entry-price of your position and you can exactly calculate the TakeProfit-limit (where your trade-profit exceeds the `tp_stop`-value). You will place at that price-level a TakeProfit-limit-order.

This order will be filled, if intra-bar any price exceeds that limit. In other words: If the high of a bar exceeds the take-profit-limit, the stop will be triggered intrabar.

With vbt we can exactly mimic this behaviour, by handing over the high-series as parameter, when generating the portfolio-instance.

In [29]:
# generate a portfolio instance only with take-profit stops, high series passed
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    # exits,                # no exits from the system
    tp_stop=0.15,           # take profit at 15%
    high=high,              # series of the high-prices from OHLC-data
    init_cash=10_000,       # initial cash
    freq='D',               # all parameters on daily base
)

For entries, the behavior remains the same:
* Entry signal occurs at bar [i]
* Entry date = date of bar [i], entry price = close [i]
* Under the hood: the take-profit limit is calculated as (1 + `tp_stop`) * close[i]

For exits, we observe a different behavior compared to the previous simulation:
* Take-profit exit conditions are fulfilled at bar [j] (in our example, this happens one bar earlier)
* Exit date = date of bar [j]
* Exit price: not directly found in OHLC data, but set to the take-profit limit calculated at entry

Note: As a result, the trade PnL is very close to the specified `tp_stop` value.

This approach effectively mimics the execution of a limit order intra-bar, triggered once the trade profit exceeds the `tp_stop` value.

In [30]:
# trades and key data
trades = pf.trades.records_readable

# display trades and key trade data
n = 3               # number of trades to be displayed
print(f'xxx  first {n} trades with key trade data')
display(trades.head(n))
print("\n")


xxx  first 3 trades with key trade data


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,67.429314,2022-07-19,148.303452,0.0,2022-08-16,170.548969,0.0,1500.000000,0.150000,Long,Closed,0
1,1,AAPL,77.272076,2022-11-23,148.824783,0.0,2023-05-05,171.251480,0.0,1732.957423,0.150692,Long,Closed,1
2,2,AAPL,71.662704,2023-11-10,184.656128,0.0,2024-06-12,212.354547,0.0,1984.943613,0.150000,Long,Closed,2


## 8. Stop Loss

We now study a StopLoss in isolation by disabling strategy-based exits and using only StopLoss exits.

We can set:
* `sl_stop` = value (the percentage of acceptable loss until the stop is triggered)
* `sl_trail` = True or False (controls trailing behavior)

If `sl_trail` is set to True, the stop loss trails the price—it increases proportionally with rising prices but remains unchanged if the price declines.

The portfolio instance using only a (trailing) StopLoss as the exit for positions is generated as follows:


In [31]:
# generate a portfolio instance only with (trailing) stop-loss
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    # exits,                # no exits from the system
    sl_stop=0.05,           # stop loss at 5%
    sl_trail=True,          # trailing stop loss
    init_cash=10_000,       # initial cash
    freq='D',               # all parameters on daily base
)

By inspecting different scenarios, we observe the following:

* Entry
    * Signal is triggered at bar [i]
    * Trade is executed at close [i] as the entry price
    * Under the hood, the initial Stop-Loss is calculated as (1 - `sl_stop`) * close [i]

* During the Trade
    * The current (trailing) Stop-Loss is updated if prices increase and `sl_trail` = True; otherwise, it remains unchanged
    * Formula: trailing_price [i] = max((1 - `sl_stop`) * price [i], trailing_price [i-1])
    * In the current (default) setup, price = close

*  Exit
    * At bar [j], a (trailing) Stop-Loss signal is triggered when close [j] falls below the current trailing Stop-Loss
    * Exit price = close [j]

Note: It is currently a bit difficult to demonstrate in detail, how and when a StopLoss is triggered, since its current value is not directly accessible with the methods presented so far.

In the next part of this blog, we will show methods to access these values via callback functions and review the details of (trailing) StopLoss behavior again.

In [32]:
# trades and key data
trades = pf.trades.records_readable

# generating timeline for standard + specific trading parameter
timeline = get_timeline_basic(prices, entries, exits, trades)    # standards (price, entry, exits)

# display trades and key trade data
n = 3               # number of trades to be displayed
print(f'xxx  first {n} trades with key trade data')
display(trades.head(n))
print("\n")

# display timeline around entry and exit of the first trade
entry_date = '2022-07-19'
exit_date = '2022-08-17'
n_r = 1             # number of rows before/aftr entry/exit
lense_at_entry_exit(timeline, entry_date, exit_date, n_r)

xxx  first 3 trades with key trade data


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,67.429314,2022-07-19,148.303452,0.0,2022-08-26,160.921341,0.0,850.815627,0.085082,Long,Closed,0
1,1,AAPL,72.910005,2022-11-23,148.824783,0.0,2022-11-29,139.071930,0.0,-711.080585,-0.065532,Long,Closed,1
2,2,AAPL,71.976948,2023-01-30,140.874756,0.0,2023-02-24,144.750290,0.0,278.949115,0.027510,Long,Closed,2




### Timeline around Entry Date: 2022-07-19 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits
Date,,,,,,,,
2022-07-18,148.048109,148.863289,144.080247,144.443649,False,False,False,False
2022-07-19,145.278452,148.529340,144.286494,148.303452,True,False,True,False
2022-07-20,148.421311,150.974887,147.684704,150.307022,False,False,False,False




### Timeline around Exit Date: 2022-08-17 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits
Date,,,,,,,,
2022-08-16,169.930240,170.844909,168.828718,170.176117,False,False,False,False
2022-08-17,169.920426,173.244668,169.723728,171.671066,False,False,False,False
2022-08-18,170.884252,172.015279,170.264638,171.277649,False,False,False,False


## 9. Advanced Stop Loss Techniques

Now we have to acknowledge a similar artificial situation (as with the take-profit). In reality, we will not wait until the close price declines below the trailing-stop and execute the order at close only.

In real life you know for each bar the (trailing)-StopLoss and you will place an StopLoss-order at the market.

Once the price intraday drops below that StopLoss-value, the exit-order is executed.

To mimic this behaviour we need to give the high- and low-price series as arguments to the `.from_signal(...)`class function.

In [33]:
# generate a portfolio instance only with trailing Stop-Loss
pf = vbt.Portfolio.from_signals(
    close,                  # price info (defines also the index of any timeline)
    entries,                # series (boolean) of entry-signals
    # exits,                # no exits from the system
    sl_stop=0.05,           # stop loss at 5%
    sl_trail=True,          # trailing stop loss
    high=high,              # series of the high-prices from OHLC-data
    low=low,                # series of the low-prices from OHLC-data
    init_cash=10_000,       # initial cash
    freq='D',               # all parameters on daily base
)

We observe:
* Entry and during trade:
    * usual behaviour (as above)
* Exits:
    * at bar [j] a trailing-stop is triggered, when ???
    * exit price is ???

Note: data, labelled '???' cannot be derived diretly from the timeline. From the available data it is not possible to decide, at what bar a (trailing)StopLoss is triggered. We will get back to this, when discussing callback-functions (and extracting the exact StopLoss and TakeProfit-values) .

FOR THE TIME BEING (without evidence, but we will prove this later), the (trailing) StopLoss mechanism is as follows:
* trailing StopLoss formula: trailing_stop [j] = max((1 - `sl_stop`) * high [j-1], trailing_stop [j-1])
* trigger of trailing stopp loss at bar [j] when low [j] < trailing_stop [j]

So the (trailing) StopLoss is calculated based on the **highs** (i.e. in a conservative manner, since it elevates the StopLoss to the highest reasonable level). The (trailing) StopLoss signal is triggered intrabar if the **low** of a bar is below the (trailing) StopLoss at that specific bar.

In [34]:
# trades and key data
trades = pf.trades.records_readable

# generating timeline for standard + specific trading parameter
timeline = get_timeline_basic(prices, entries, exits, trades)    # standards (price, entry, exits)

# display trades and key trade data
n = 3               # number of trades to be displayed
print(f'xxx  first {n} trades with key trade data')
display(trades.head(n))
print("\n")

# display timeline around entry and exit of the first trade
entry_date = '2022-07-19'
exit_date = '2022-08-22'
n_r = 1             # number of rows before/aftr entry/exit
lense_at_entry_exit(timeline, entry_date, exit_date, n_r)

xxx  first 3 trades with key trade data


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,AAPL,67.429314,2022-07-19,148.303452,0.0,2022-08-22,164.582434,0.0,1097.680642,0.109768,Long,Closed,0
1,1,AAPL,74.568767,2022-11-23,148.824783,0.0,2022-11-28,141.383544,0.0,-554.884032,-0.050000,Long,Closed,1
2,2,AAPL,74.838083,2023-01-30,140.874756,0.0,2023-02-10,147.288948,0.0,480.025861,0.045531,Long,Closed,2




### Timeline around Entry Date: 2022-07-19 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits
Date,,,,,,,,
2022-07-18,148.048109,148.863289,144.080247,144.443649,False,False,False,False
2022-07-19,145.278452,148.529340,144.286494,148.303452,True,False,True,False
2022-07-20,148.421311,150.974887,147.684704,150.307022,False,False,False,False




### Timeline around Exit Date: 2022-08-22 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits
Date,,,,,,,,
2022-08-19,170.176129,170.874426,168.484497,168.691040,False,False,False,False
2022-08-22,166.891212,167.058406,164.383267,164.806183,False,False,False,True
2022-08-23,164.324267,165.927387,163.901351,164.471786,False,False,False,False


## 10. Combining Exits from Strategy, Take-Profits and Stop Losses

Now we can put all different type of exits together:
* exit triggered by an effective exit signal from the strategy
* exit triggered by exceeding a TakeProfit-Limit
* exit triggered by underrunning a (trailing) StopLoss

The portfolio instance is generated as follows:


In [35]:
# stop parameter
stop_loss = 0.05            # 5%
stop_loss_trail = True      # trailing-stop-loss
take_profit = 0.1           # 10%

# generate a portfolio instance with all three types of exits
pf = vbt.Portfolio.from_signals(
    close,                      # price info (defines also the index of any timeline)
    entries,                    # series (boolean) of entry-signals
    exits,                      # series (boolean) of entry-signals
    sl_stop=stop_loss,          # stop loss at 5%
    sl_trail=stop_loss_trail,   # trailing stop loss
    tp_stop=take_profit,        # take profit at 15%
    init_cash=10_000,           # initial cash
    freq='D',                   # all parameters on daily base
)

Unfortunately, vbt does not deliver in `pf.trades.reports_readable` any information about the type of the effective exit from a position.

Therefore we have to add this by using of a customized  function `classify_exit_types(...)`



In the timeline, we will add a flag for the position. We start by using the method pf.net_exposure(), which returns the ratio of 'value in position' to 'total value'.

By taking the sign of this ratio, we get:
* 1 if the position is Long
* 0 if the position is Flat

This is exactly the behavior we need.

In [36]:
# list of trades with key data, amended by the trade type
trades = classify_exit_types(
    trades=pf.trades.records_readable,
    take_profit=take_profit,
    exits=exits
)

# generating timeline for standard + specific trading parameter
timeline=get_timeline_extended(prices, entries, exits, trades)    # standards (price, entry, exits)
timeline['sma_fast'] = fast_sma                                   # add indicators
timeline['sma_slow'] = slow_sma
timeline['position'] = np.sign(pf.net_exposure())                 # adding position (1 = LONG, 0 = FLAT)

We see now in the trade-table also the type of exits.

In [37]:
trades

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id,exit_type
0,0,AAPL,67.429314,2022-07-19,148.303452,0.0,2022-08-03,163.163300,0.0,1001.989358,0.100199,Long,Closed,0,take_profit
1,1,AAPL,73.925788,2022-11-23,148.824783,0.0,2022-11-29,139.071930,0.0,-720.987371,-0.065532,Long,Closed,1,stop_loss
2,2,AAPL,72.979732,2023-01-30,140.874756,0.0,2023-02-24,144.750290,0.0,282.835439,0.027510,Long,Closed,2,stop_loss
3,3,AAPL,57.208161,2023-11-10,184.656128,0.0,2024-01-02,183.903229,0.0,-43.071977,-0.004077,Long,Closed,3,stop_loss
4,4,AAPL,54.411943,2024-01-23,193.353973,0.0,2024-01-31,182.674820,0.0,-581.073492,-0.055231,Long,Closed,4,stop_loss
5,5,AAPL,54.836245,2024-05-08,181.261353,0.0,2024-06-11,205.752533,0.0,1343.004373,0.135115,Long,Closed,5,take_profit
6,6,AAPL,49.901896,2024-09-24,226.097549,0.0,2024-11-01,221.662521,0.0,-221.316308,-0.019616,Long,Closed,6,stop_loss
7,7,AAPL,48.521433,2024-11-20,227.968948,0.0,2024-12-17,252.338730,0.0,1182.456716,0.106900,Long,Closed,7,take_profit
8,8,AAPL,49.719565,2025-02-24,246.257919,0.0,2025-03-07,238.255295,0.0,-397.887013,-0.032497,Long,Closed,8,system
9,9,AAPL,55.878790,2025-07-02,211.993668,0.0,2025-08-01,201.954819,0.0,-560.958733,-0.047354,Long,Closed,9,stop_loss


In preparation for further analyses, we have also developed a very flexible plotting function `plot_multi_subplot_trading_data(...)`. The source code is not included here but can be inspected on GitHub.

This function leverages standard Plotly functionality and allows the creation of customized, flexible dashboards.

Below is the first example layout:
* Upper plot: Candlestick price chart with indicators, entries, and exits (by type)
* Lower plot: Position

In [38]:
# plot configuration 01:
# 1. Subplot: Candlesticks, SMA's, effective Entry/Exit-Signals;
# 2. Subplot: Positions

subplot_config_01 = [
    {"title": "Prices, Indicators, effective Signals", "height_ratio": 8,
     "traces": [
          {"type": "candlestick", "open": "open", "high": "high", "low": "low", "close": "close"},
          {"type": "scatter", "y": "sma_fast", "name": "SMA Fast", "line": {"color": "orange"}},
          {"type": "scatter", "y": "sma_slow", "name": "SMA Slow", "line": {"color": "green"}},
          {"type": "scatter_markers", "y": "eff_entries", "marker": {"color": "green", "symbol": "triangle-up", "size": 15}},
          {"type": "scatter_markers", "y": "eff_exits_system", "marker": {"color": "red", "symbol": "triangle-down", "size": 15}},
          {"type": "scatter_markers", "y": "eff_exits_stop_loss", "marker": {"color": "blue", "symbol": "triangle-down", "size": 15}},
          {"type": "scatter_markers", "y": "eff_exits_take_profit", "marker": {"color": "grey", "symbol": "triangle-down", "size": 15}},
        ],
    },
    { "title": "Position", "height_ratio": 1,
     "traces": [
         {"type": "scatter", "y": "position", "name": "Position", "line": {"color": "green", "width": 2},
          "line_shape": "hv",   # <-- stepped line
          "mode": "lines"       # ensure 'lines' mode is used
          }
         ],
    },
]

In [39]:
fig = plot_multi_subplot_trading_data(
    timeline,
    subplot_config_01,
    title = f'Overview for {ticker}, from {start_date} to {end_date}',
    size=(1400, 500),
    theme='plotly',
)
fig.show()